# This notebook contains code that can be used to create a custom taxonomy from a phylogenetic tree that can be used for Kraken or emboss

In [6]:
# Load the phylogenetic tree into memory
from ete3 import Tree
from os import chdir, path, makedirs, system
from Bio.SeqIO import index as fasta_index,write

chdir("/mnt/research/germs/shane/hgt/data")
prot="Cas9"
# phyloTree = Tree("trees/RAxML_bestTree.Cas12_FA")
phyloTree = Tree("trees/RAxML_bestTree.Cas9_98_AVX2")
chdir("/mnt/research/germs/shane/antibioticResistance/data")

In [7]:
specificDB = "dbs/cas9_Operons/taxonomy"
if not path.exists(specificDB): makedirs(specificDB);print("Made "+specificDB)

In [3]:
trees = [("RAxML_bestTree.Cas9_98_AVX2","Cas9"),("RAxML_result.Cas12_98_AVX3","Cas12"),("RAxML_result.Cas2_98_AVX3","Cas2")] #other trees to connect
# trees/RAxML_bestTree.Cas1_98_AVX2

In [8]:
allSeqs = fasta_index("assemblies/Cas9CodingOperons.fa","fasta")

In [9]:
# Generate the custom taxonomy for Kraken based on the loaded phylogenetic tree
nodesTracker={}
names= open(specificDB+"/names.dmp",'w')
nodes= open(specificDB+"/nodes.dmp",'w')
root = phyloTree.get_tree_root()
allLeaves={}
for taxID,node in enumerate(phyloTree.traverse("preorder")):
    nodesTracker[node]=str(taxID+1)
    if not node.is_leaf():
        node.name = str(taxID+1)
        if node ==root:
            node.name = 'root'
            nodes.write("\t|\t".join([str(taxID+1),'1',"no rank","-"])+'\n') # Root is its own parent
        else: 
            nodes.write("\t|\t".join([str(taxID+1),nodesTracker[node.up],"no rank","-"])+'\n')  
    else: 
        nodes.write("\t|\t".join([str(taxID+1),nodesTracker[node.up],"species","-"])+'\n')
    if not node.is_leaf():names.write("\t|\t".join([str(taxID+1),"Cas9_"+node.name,"-","scientific name",""])+'\n')
    else:
        names.write("\t|\t".join([str(taxID+1),node.name,"-","scientific name",""])+'\n')
        allLeaves[node.name]=str(taxID+1)
        count = 0
        altID = node.name +"_%i" % (count)
        while altID in allSeqs:
            allLeaves[altID]=str(taxID+1)
            names.write("\t|\t".join([str(taxID+1),altID,"-","scientific name",""])+'\n')
            count+=1
            altID = node.name +"_%i" % (count)
    
print(len(phyloTree))
names.close()
nodes.close()

2307


In [5]:
# allSeqs = fasta_index("assemblies/Cas9CodingORFs.fa","fasta")
missing,good=0,0
with open("assemblies/Cas12CodingOperonsCleaned.fa",'w') as fh:
    for leaf in phyloTree.get_leaves():
        if leaf.name not in allSeqs: missing+=1;continue
        
        rec=allSeqs[leaf.name]
        rec.id = rec.id + "|kraken:taxid|" + nodesTracker[leaf]+ "\t%s ORF Region" % (prot)
        rec.name = rec.id
        rec.description = ""
        if str(rec.seq)=="":
            print(rec.id)
        write(rec,fh,"fasta")
        del allLeaves[leaf.name]
#         c=0
#         seqIDClusterID = leaf.name +"_%i" % (c)
#         while seqIDClusterID in allSeqs:
#             rec=allSeqs[seqIDClusterID]
#             rec.id = rec.id + "|kraken:taxid|" + nodesTracker[leaf]+ "\t%s ORF Region" % (prot)
#             rec.name = rec.id
#             rec.description = ""
#             if str(rec.seq)=="":
#                 print(rec.id)
#             write(rec,fh,"fasta")
#             c+=1
#             seqIDClusterID = leaf.name +"_%i" % (c)
#             good+=1
            
        good+=1
print(missing,good,sep='\t')

0	1834


In [6]:
with open("assemblies/Cas12CodingOperonsCleaned.fa",'a') as fh:
    for sid in allLeaves:
        rec=allSeqs[sid]
        rec.id = rec.id + "|kraken:taxid|" + allLeaves[sid]+ "\t%s ORF Region" % (prot)
        rec.name = rec.id
        rec.description = ""
        if str(rec.seq)=="":
            print(rec.id)
        write(rec,fh,"fasta")
        good+=1
good
        

3454

In [5]:
phyloTrees = {"RAxML_bestTree.Cas1_98_AVX2":phyloTree}

In [6]:
taxIDCount = taxID+1
for treeFile,prot in trees:
    phyloTrees[treeFile] = Tree("/mnt/research/germs/shane/hgt/data/trees/"+treeFile)
    phyloTree = phyloTrees[treeFile]
    root = phyloTree.get_tree_root()
    for i,node in enumerate(phyloTree.traverse("preorder")):
        taxID=taxIDCount+i
        nodesTracker[node]=str(taxID+1)
        if not node.is_leaf():
            node.name = str(taxID+1)
            if node ==root:nodes.write("\t|\t".join([str(taxID+1),'1',"no rank","-"])+'\n') # Root is its own parent
            else: nodes.write("\t|\t".join([str(taxID+1),nodesTracker[node.up],"no rank","-"])+'\n')
        else: nodes.write("\t|\t".join([str(taxID+1),nodesTracker[node.up],"species","-"])+'\n')
        names.write("\t|\t".join([str(taxID+1),prot+"_"+node.name,"-","scientific name",""])+'\n')
    print(len(phyloTree))
names.close()
nodes.close()

2307
1834
1722


In [10]:
# Rewrite the fasta file with the taxonomy IDs generated in Cell 2 so that Kraken can link taxonomy to the sequences
trees = [("RAxML_bestTree.Cas1_98_AVX2", "Cas1",  "Cas1_Class2_Coding.fa"),
         ("RAxML_bestTree.Cas9_98_AVX2", "Cas9",  "Cas9Coding.fa"),
         ("RAxML_result.Cas12_98_AVX3",  "Cas12", "Cas12Coding.fa"),
         ("RAxML_result.Cas2_98_AVX3",   "Cas2",  "Cas2Coding.fa")]

for treeFile, prot, seqFile in trees:
    phyloTree = phyloTrees[treeFile]
    allSeqs = fasta_index("assemblies/"+seqFile,"fasta")
    cleanedFile = seqFile.replace(".fa","Cleaned.fa")
    missing,good=0,0
    with open("assemblies/"+cleanedFile,'w') as fh:
        for leaf in phyloTree.get_leaves():
            if leaf.name not in allSeqs: missing+=1;continue
            rec=allSeqs[leaf.name]
            rec.id = rec.id + "|kraken:taxid|" + nodesTracker[leaf]+ "\t%s ORF Region" % (prot)
            rec.name = rec.id
            rec.description = ""
            if str(rec.seq)=="":
                print(rec.id)
            write(rec,fh,"fasta")
            good+=1
    print(prot,missing,good,"assemblies/"+cleanedFile,sep='\t')

Cas1	0	2184	assemblies/Cas1_Class2_CodingCleaned.fa
Cas9	0	2307	assemblies/Cas9CodingCleaned.fa
Cas12	0	1834	assemblies/Cas12CodingCleaned.fa
Cas2	0	1722	assemblies/Cas2CodingCleaned.fa


In [14]:
%%bash 
#Add the sequences that with the added taxIDs generated by this module to a custom database
kraken2-build --add-to-library assemblies/Cas1_Class2_CodingCleaned.fa --db dbs/CRISPRCas_DB
kraken2-build --add-to-library assemblies/Cas9CodingCleaned.fa --db dbs/CRISPRCas_DB
kraken2-build --add-to-library assemblies/Cas12CodingORFsCleaned.fa --db dbs/CRISPRCas_DB
kraken2-build --add-to-library assemblies/Cas2CodingCleaned.fa --db dbs/CRISPRCas_DB

Masking low-complexity regions of new file...Error: (106.16) Application's execution failed (CObjReaderParseException::eNoResidues) FASTA-Reader: No residues given (m_Pos = 37159)
Masking low-complexity regions of new file... done.
Added "assemblies/Cas9Coding.fa" to library (dbs/CRISPRCas_DB)
Masking low-complexity regions of new file... done.
Added "assemblies/Cas12Coding.fa" to library (dbs/CRISPRCas_DB)
Masking low-complexity regions of new file... done.
Added "assemblies/Cas2Coding.fa" to library (dbs/CRISPRCas_DB)


In [ ]:
%%bash
#Build the Kraken database
kraken2-build --threads 15 --build --db dbs/CRISPRCas_DB

In [ ]:
%%bash
# Build the bracken database with kmer lengths=31 and mean read length of my data is 100bp (-l 100)
~/bin/Bracken-2.5/bracken-build -k 31 -l 100 -d dbs/CRISPRCas_DB -t 10